In [1]:
#!/usr/bin/env python3
"""
Script to generate a balanced synthetic dataset for the login-behavior model.
Creates ~400,000 rows (200,000 benign, 200,000 attack) with 7 behavioral features
and a Label column (0=Benign, 1=Attack). Saves to balanced_login_data.csv.
"""

import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Configuration
N_PER_CLASS = 200_000  # 200k benign + 200k attack
OUTPUT_PATH = "balanced_login_data.csv"

# Feature distributions
USER_AGENTS = [
    # Human-like
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0",
    # Bot-like
    "curl/8.5.0",
    "python-requests/2.28.1",
    "go-http-client/1.1",
    "Wget/1.20.3",
    "Scrapy/2.7.0"
]
IP_POOL = [
    "203.0.113.1", "203.0.113.2", "45.77.1.3", "51.15.39.9", "192.0.2.1",
    # Additional IPs for diversity
    "192.168.1.100", "10.0.0.50", "172.16.0.10"
]

# Generate data
data = {
    "time_to_submit": np.concatenate([
        # Benign: Human-like, 1.5–12s with slight Gaussian noise
        np.random.uniform(1.5, 12, N_PER_CLASS) + np.random.normal(0, 0.2, N_PER_CLASS),
        # Attack: Bot-like, 0.02–0.6s
        np.random.uniform(0.02, 0.6, N_PER_CLASS) + np.random.normal(0, 0.05, N_PER_CLASS)
    ]),
    "user_agent": np.concatenate([
        # Benign: Mostly human-like browsers (80%), some bot-like (20%)
        np.random.choice(USER_AGENTS, N_PER_CLASS, p=[0.3, 0.3, 0.2, 0.05, 0.05, 0.05, 0.025, 0.025]),
        # Attack: Mostly bot-like (80%), some human-like (20%)
        np.random.choice(USER_AGENTS, N_PER_CLASS, p=[0.05, 0.05, 0.1, 0.3, 0.2, 0.2, 0.05, 0.05])
    ]),
    "login_hour": np.random.choice(range(24), 2 * N_PER_CLASS),  # Uniform for both
    "client_ip": np.random.choice(IP_POOL, 2 * N_PER_CLASS),  # Uniform for both
    "password_length": np.random.choice(range(6, 16), 2 * N_PER_CLASS),  # Uniform 6–15
    "failed_login_count_last_10min": np.concatenate([
        # Benign: Low fails (Poisson mean 0.4)
        np.random.poisson(0.4, N_PER_CLASS),
        # Attack: High fails (Poisson mean 3.5)
        np.random.poisson(3.5, N_PER_CLASS)
    ]),
    "is_username_email": np.random.choice([0, 1], 2 * N_PER_CLASS),  # Uniform for both
    "label": np.concatenate([
        np.zeros(N_PER_CLASS, dtype=int),  # Benign
        np.ones(N_PER_CLASS, dtype=int)   # Attack
    ])
}

# Create DataFrame
df = pd.DataFrame(data)

# Clip time_to_submit to ensure no negative values
df["time_to_submit"] = df["time_to_submit"].clip(lower=0.01)

# Shuffle rows
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Validate
print("Dataset Shape:", df.shape)
print("\nLabel Counts:\n", df["label"].value_counts())
print("\nFeature Summary:\n", df.describe(include="all"))
print("\nMissing Values:\n", df.isnull().sum())

# Save to CSV
df.to_csv(OUTPUT_PATH, index=False)
print(f"\n✅ Saved to {OUTPUT_PATH}")

Dataset Shape: (400000, 8)

Label Counts:
 label
0    200000
1    200000
Name: count, dtype: int64

Feature Summary:
         time_to_submit                                         user_agent  \
count    400000.000000                                             400000   
unique             NaN                                                  8   
top                NaN  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...   
freq               NaN                                              70339   
mean          3.532361                                                NaN   
std           3.872805                                                NaN   
min           0.010000                                                NaN   
25%           0.309767                                                NaN   
50%           0.838234                                                NaN   
75%           6.761922                                                NaN   
max          12.683748             

In [2]:
#!/usr/bin/env python3
"""
Script to train a LightGBM model on balanced_login_data.csv for login-behavior classification.
Encodes categorical features, splits data (90% train, 10% validation), tunes hyperparameters
with Optuna, and saves the model as app/models/login_classifier.pkl.
"""

import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import optuna

# Configuration
DATA_PATH = "balanced_login_data.csv"
MODEL_PATH = "app/models/login_classifier.pkl"
VALIDATION_SIZE = 0.1
N_TRIALS = 30  # Number of Optuna trials for hyperparameter tuning
RANDOM_SEED = 42

# Set random seed
np.random.seed(RANDOM_SEED)

# 1. Load and preprocess data
df = pd.read_csv(DATA_PATH)

# Encode categorical features
le_user_agent = LabelEncoder()
le_client_ip = LabelEncoder()
df["user_agent"] = le_user_agent.fit_transform(df["user_agent"])
df["client_ip"] = le_client_ip.fit_transform(df["client_ip"])

# Features and labels
feature_cols = [
    "time_to_submit",
    "user_agent",
    "login_hour",
    "client_ip",
    "password_length",
    "failed_login_count_last_10min",
    "is_username_email",
]
X = df[feature_cols]
y = df["label"]

# Split data (90% train, 10% validation, stratified)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=VALIDATION_SIZE, stratify=y, random_state=RANDOM_SEED
)

# 2. Define Optuna objective for hyperparameter tuning
def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 16, 128),
        "max_depth": trial.suggest_int("max_depth", 4, 16),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 100),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),
        "scale_pos_weight": 1.0,  # Balanced dataset
    }
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val)
    booster = lgb.train(
        params,
        dtrain,
        valid_sets=[dval],
        num_boost_round=200,
        callbacks=[lgb.early_stopping(stopping_rounds=20), lgb.log_evaluation(0)],
    )
    preds = booster.predict(X_val)
    auc = roc_auc_score(y_val, preds)
    return auc

# 3. Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)
print("Best params:", study.best_params)

# 4. Train final model with best parameters
best_params = study.best_params
best_params.update({"objective": "binary", "metric": "auc", "verbosity": -1})
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_val, label=y_val)
booster = lgb.train(
    best_params,
    dtrain,
    valid_sets=[dval],
    num_boost_round=200,
    callbacks=[lgb.early_stopping(stopping_rounds=20), lgb.log_evaluation(20)],
)

# 5. Evaluate on validation set with multiple thresholds
pred_probs = booster.predict(X_val)
thresholds = [0.5, 0.4, 0.3]
for threshold in thresholds:
    preds = (pred_probs >= threshold).astype(int)
    print(f"\n=== Results for threshold={threshold} ===")
    print("AUC:", roc_auc_score(y_val, pred_probs))
    print("Confusion Matrix:\n", confusion_matrix(y_val, preds))
    print("Classification Report:\n", classification_report(y_val, preds))

# 6. Feature importance
importance = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": booster.feature_importance(importance_type="gain"),
})
importance = importance.sort_values("Importance", ascending=False)
print("\nFeature Importance:\n", importance)

# 7. Wrap model in sklearn-compatible interface
class LGBMWrapper:
    def __init__(self, booster, feature_names, le_user_agent, le_client_ip):
        self.booster = booster
        self.feature_names = feature_names
        self.le_user_agent = le_user_agent
        self.le_client_ip = le_client_ip

    def predict_proba(self, X):
        # Handle DataFrame input
        if isinstance(X, pd.DataFrame):
            X = X.copy()
            if "user_agent" in X.columns:
                X["user_agent"] = self.le_user_agent.transform(X["user_agent"])
            if "client_ip" in X.columns:
                X["client_ip"] = self.le_client_ip.transform(X["client_ip"])
            X = X[self.feature_names].values.astype(np.float32)
        else:
            X = np.asarray(X, dtype=np.float32)
        p1 = self.booster.predict(X)
        p0 = 1.0 - p1
        return np.vstack([p0, p1]).T

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= threshold).astype(int)

# 8. Save model
clf = LGBMWrapper(booster, feature_cols, le_user_agent, le_client_ip)
os.makedirs("app/models", exist_ok=True)
joblib.dump(clf, MODEL_PATH)
print(f"\n✅ Saved model to {MODEL_PATH}")

d:\Canada\Subjects\Semester -2\AIDI-2005-02 CAPSTONE TERM ll\Bot_detector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-07-17 11:16:46,810] A new study created in memory with name: no-name-3bfd8343-d857-458f-b2c9-00548d2d89b3


Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[38]	valid_0's auc: 1


[I 2025-07-17 11:16:49,712] Trial 0 finished with value: 0.9999999075 and parameters: {'learning_rate': 0.16755889233749574, 'num_leaves': 16, 'max_depth': 13, 'feature_fraction': 0.6972529424542786, 'bagging_fraction': 0.8018172797108292, 'bagging_freq': 4, 'min_data_in_leaf': 33, 'lambda_l1': 2.480643012181809, 'lambda_l2': 0.8682096696093994}. Best is trial 0 with value: 0.9999999075.


Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[96]	valid_0's auc: 1


[I 2025-07-17 11:16:54,237] Trial 1 finished with value: 0.999999905 and parameters: {'learning_rate': 0.08197881344662493, 'num_leaves': 40, 'max_depth': 7, 'feature_fraction': 0.6130251396579516, 'bagging_fraction': 0.5480146493991442, 'bagging_freq': 9, 'min_data_in_leaf': 89, 'lambda_l1': 0.16368394435161027, 'lambda_l2': 3.7726843362704776}. Best is trial 0 with value: 0.9999999075.


...

Early stopping, best iteration is:
[42]	valid_0's auc: 1
Training until validation scores don't improve for 20 rounds


[I 2025-07-17 11:17:07,023] Trial 28 finished with value: 0.99999932625 and parameters: {'learning_rate': 0.10459763514163894, 'num_leaves': 115, 'max_depth': 15, 'feature_fraction': 0.7954140887680629, 'bagging_fraction': 0.6472013778639256, 'bagging_freq': 7, 'min_data_in_leaf': 38, 'lambda_l1': 2.301260470527981, 'lambda_l2': 3.0059507146883675}. Best is trial 16 with value: 0.9999999199999999.


Early stopping, best iteration is:
[1]	valid_0's auc: 0.999999
Training until validation scores don't improve for 20 rounds


[I 2025-07-17 11:17:07,293] Trial 29 finished with value: 0.99999991 and parameters: {'learning_rate': 0.16398881909552104, 'num_leaves': 89, 'max_depth': 13, 'feature_fraction': 0.9971661139039512, 'bagging_fraction': 0.7768452992905411, 'bagging_freq': 6, 'min_data_in_leaf': 27, 'lambda_l1': 2.4126469095911363, 'lambda_l2': 4.028260777047949}. Best is trial 16 with value: 0.9999999199999999.


Early stopping, best iteration is:
[26]	valid_0's auc: 1
Best params: {'learning_rate': 0.16296242250101053, 'num_leaves': 75, 'max_depth': 15, 'feature_fraction': 0.9199843192078083, 'bagging_fraction': 0.6271157784617101, 'bagging_freq': 5, 'min_data_in_leaf': 34, 'lambda_l1': 0.0009722459428442853, 'lambda_l2': 4.9156352459893995}
Training until validation scores don't improve for 20 rounds
[20]	valid_0's auc: 1
[40]	valid_0's auc: 1
[60]	valid_0's auc: 1
Early stopping, best iteration is:
[40]	valid_0's auc: 1

=== Results for threshold=0.5 ===
AUC: 0.9999999199999999
Confusion Matrix:
 [[19996     4]
 [    5 19995]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     20000
           1       1.00      1.00      1.00     20000

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000


=== Results for thr

In [1]:
#!/usr/bin/env python3
"""
Script to train a slim LightGBM model on balanced_login_data.csv using the top 3 features
(time_to_submit, failed_login_count_last_10min, user_agent). Saves the model as
app/models/slim_login_classifier.pkl.
"""

import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# Configuration
DATA_PATH = "balanced_login_data.csv"
MODEL_PATH = "app/models/slim_login_classifier.pkl"
VALIDATION_SIZE = 0.1
RANDOM_SEED = 42
BEST_PARAMS = {
    "learning_rate": 0.16296242250101053,
    "num_leaves": 75,
    "max_depth": 15,
    "feature_fraction": 0.9199843192078083,
    "bagging_fraction": 0.6271157784617101,
    "bagging_freq": 5,
    "min_data_in_leaf": 34,
    "lambda_l1": 0.0009722459428442853,
    "lambda_l2": 4.9156352459893995,
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
}

# Set random seed
np.random.seed(RANDOM_SEED)

# 1. Load and preprocess data
df = pd.read_csv(DATA_PATH)

# Encode categorical feature
le_user_agent = LabelEncoder()
df["user_agent"] = le_user_agent.fit_transform(df["user_agent"])

# Select top 3 features
feature_cols = [
    "time_to_submit",
    "failed_login_count_last_10min",
    "user_agent",
]
X = df[feature_cols]
y = df["label"]

# Split data (90% train, 10% validation, stratified)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=VALIDATION_SIZE, stratify=y, random_state=RANDOM_SEED
)

# 2. Train model with best parameters
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_val, label=y_val)
booster = lgb.train(
    BEST_PARAMS,
    dtrain,
    valid_sets=[dval],
    num_boost_round=200,
    callbacks=[lgb.early_stopping(stopping_rounds=20), lgb.log_evaluation(20)],
)

# 3. Evaluate on validation set with multiple thresholds
pred_probs = booster.predict(X_val)
thresholds = [0.5, 0.4, 0.3]
for threshold in thresholds:
    preds = (pred_probs >= threshold).astype(int)
    print(f"\n=== Results for threshold={threshold} ===")
    print("AUC:", roc_auc_score(y_val, pred_probs))
    print("Confusion Matrix:\n", confusion_matrix(y_val, preds))
    print("Classification Report:\n", classification_report(y_val, preds))

# 4. Feature importance
importance = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": booster.feature_importance(importance_type="gain"),
})
importance = importance.sort_values("Importance", ascending=False)
print("\nFeature Importance:\n", importance)

# 5. Wrap model in sklearn-compatible interface
class LGBMWrapper:
    def __init__(self, booster, feature_names, le_user_agent):
        self.booster = booster
        self.feature_names = feature_names
        self.le_user_agent = le_user_agent

    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.copy()
            if "user_agent" in X.columns:
                X["user_agent"] = self.le_user_agent.transform(X["user_agent"])
            X = X[self.feature_names].values.astype(np.float32)
        else:
            X = np.asarray(X, dtype=np.float32)
        p1 = self.booster.predict(X)
        p0 = 1.0 - p1
        return np.vstack([p0, p1]).T

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= threshold).astype(int)

# 6. Save model
clf = LGBMWrapper(booster, feature_cols, le_user_agent)
os.makedirs("app/models", exist_ok=True)
joblib.dump(clf, MODEL_PATH)
print(f"\n✅ Saved model to {MODEL_PATH}")

Training until validation scores don't improve for 20 rounds
[20]	valid_0's auc: 1
[40]	valid_0's auc: 1
[60]	valid_0's auc: 1
Early stopping, best iteration is:
[43]	valid_0's auc: 1

=== Results for threshold=0.5 ===
AUC: 0.9999999175000001
Confusion Matrix:
 [[19996     4]
 [    5 19995]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     20000
           1       1.00      1.00      1.00     20000

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000


=== Results for threshold=0.4 ===
AUC: 0.9999999175000001
Confusion Matrix:
 [[19996     4]
 [    1 19999]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     20000
           1       1.00      1.00      1.00     20000

    accuracy                           1.00     40000
   ma

In [8]:
#!/usr/bin/env python3
"""
Script to create an ensemble classifier combining the slim login-behavior model
and network-flow model. Generates a synthetic test dataset, evaluates the ensemble
with adjustable threshold and logic, and saves it as app/models/ensemble_classifier.pkl.
"""

import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import os
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Configuration
LOGIN_MODEL_PATH = "app/models/slim_login_classifier.pkl"
NETWORK_MODEL_PATH = "D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\notebooks\\final_lightgbm_tuned.txt"
ENSEMBLE_MODEL_PATH = "app/models/ensemble_classifier.pkl"
TEST_SIZE = 10000
RANDOM_SEED = 42
THRESHOLD = 0.5  # Increased to 0.5 for better balance
USE_MAX_PROB = False  # Set to True for max probability rule instead of averaging

# Set random seed
np.random.seed(RANDOM_SEED)

# 1. Generate synthetic test dataset
def generate_synthetic_test_data(n_samples=TEST_SIZE):
    n_benign = n_samples // 2
    n_attack = n_samples - n_benign
    data = {
        # Login-behavior features (from balanced_login_data.csv distributions)
        "time_to_submit": np.concatenate([
            np.random.uniform(1.5, 12.0, n_benign),  # Benign: 1.5-12s
            np.random.uniform(0.02, 0.6, n_attack)   # Attack: 0.02-0.6s
        ]),
        "failed_login_count_last_10min": np.concatenate([
            np.random.randint(0, 3, n_benign),       # Benign: 0-2
            np.random.randint(3, 16, n_attack)       # Attack: 3-15
        ]),
        "user_agent": np.concatenate([
            np.random.choice(
                ["Mozilla/5.0 (Windows NT 10.0; Win64; x64)", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)", "Chrome/90.0"],
                n_benign, p=[0.4, 0.3, 0.3]
            ),
            np.random.choice(
                ["curl/8.5.0", "python-requests/2.28.1", "BotAgent"],
                n_attack, p=[0.5, 0.3, 0.2]
            )
        ]),
        # Placeholder network-flow features (78 features, simplified distributions)
        **{
            f"flow_feature_{i}": np.random.uniform(0, 100, n_samples) for i in range(78)
        },
        "label": np.concatenate([np.zeros(n_benign), np.ones(n_attack)])
    }
    df = pd.DataFrame(data)
    return df

# 2. Load models
login_model = joblib.load(LOGIN_MODEL_PATH)
network_model = lgb.Booster(model_file=NETWORK_MODEL_PATH)

# 3. Ensemble classifier
class EnsembleClassifier:
    def __init__(self, login_model, network_model, login_features, network_features, threshold=0.3):
        self.login_model = login_model
        self.network_model = network_model
        self.login_features = login_features
        self.network_features = network_features
        self.threshold = threshold
        self.le_user_agent = LabelEncoder()  # Initialize new LabelEncoder

    def fit_encoders(self, X):
        # Fit LabelEncoder on user_agent values from the input data
        self.le_user_agent.fit(X["user_agent"])

    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.copy()
            if "user_agent" in X.columns:
                X["user_agent"] = self.le_user_agent.transform(X["user_agent"])
            X_login = X[self.login_features].values.astype(np.float32)
            X_network = X[self.network_features].values.astype(np.float32)
        else:
            X_login = X[:, :len(self.login_features)].astype(np.float32)
            X_network = X[:, len(self.login_features):].astype(np.float32)
        login_probs = self.login_model.predict_proba(X_login)[:, 1]
        network_probs = self.network_model.predict(X_network)
        if USE_MAX_PROB:
            ensemble_probs = np.maximum(login_probs, network_probs)
        else:
            ensemble_probs = (login_probs + network_probs) / 2
        return np.vstack([1 - ensemble_probs, ensemble_probs]).T

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= self.threshold).astype(int)

# 4. Generate and preprocess test data
test_df = generate_synthetic_test_data()
login_features = ["time_to_submit", "failed_login_count_last_10min", "user_agent"]
network_features = [f"flow_feature_{i}" for i in range(78)]
X_test = test_df[login_features + network_features]
y_test = test_df["label"]

# 5. Create and evaluate ensemble
ensemble = EnsembleClassifier(login_model, network_model, login_features, network_features, THRESHOLD)
ensemble.fit_encoders(X_test)  # Fit LabelEncoder on test data
pred_probs = ensemble.predict_proba(X_test)[:, 1]
preds = ensemble.predict(X_test)

# 6. Print evaluation metrics
print("\n=== Ensemble Results ===")
print("AUC:", roc_auc_score(y_test, pred_probs))
print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
print("Classification Report:\n", classification_report(y_test, preds))

# 7. Save ensemble model
os.makedirs("app/models", exist_ok=True)
joblib.dump(ensemble, ENSEMBLE_MODEL_PATH)
print(f"\n✅ Saved ensemble model to {ENSEMBLE_MODEL_PATH}")


=== Ensemble Results ===
AUC: 1.0
Confusion Matrix:
 [[5000    0]
 [   0 5000]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5000
         1.0       1.00      1.00      1.00      5000

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000


✅ Saved ensemble model to app/models/ensemble_classifier.pkl
